In [ ]:
# default_exp components

# Components

> Supplies components that can be combinded into a dashboard or used standalone.

In [ ]:
#export
from typing import Optional, Union

from icevision.core.class_map import ClassMap

import panel as pn
import panel.widgets as pnw
from bokeh.plotting import figure
from bokeh.palettes import viridis, cividis, turbo
import numpy as np
import pandas as pd

from icevision_dashboard.utils import *

In [ ]:
pn.extension()

## Test data setup

In [ ]:
import icedata

In [ ]:
test_data_dir = icedata.fridge.load_data()
test_class_map = icedata.fridge.class_map()
test_parser = icedata.fridge.parser(test_data_dir, test_class_map)
test_train_records, test_valid_records = test_parser.parse()

In [ ]:
test_record_dataset = RecordDataset(test_valid_records, test_class_map)
test_record_dataset.data.head()

In [ ]:
test_record_dataset_no_class_map = RecordDataset(test_valid_records)
test_record_dataset_no_class_map.data.head()

## Gallery

In [ ]:
#export
def gallery(
    records,
    class_map: Optional[ClassMap] = None,
    display_label: bool = True,
    display_bbox: bool = True,
    display_mask: bool = True,
    display_keypoints: bool = True,
    width=500,
    height=500
):
    """Shows a gallery for a list of records."""
    # gui
    btn_prev = pnw.Button(name="<", width=int(2*width/6))
    btn_next = pnw.Button(name=">", width=int(2*width/6))
    current = pnw.TextInput(value="1", width=int(width/6))
    overall = pn.Row("/" + str(len(records)), width=int(width/6))
    gui = pn.Row(btn_prev, current, overall, btn_next, align="center", height=50)
    
    # plotting function
    @pn.depends(current.param.value)
    def _plot(_):
        nonlocal current
        try:
            index = int(current.value) - 1
        except:
            pass
        img  = draw_record_with_bokeh(
            record=records[index],
            class_map=class_map,
            display_label=display_label,
            display_bbox=display_bbox,
            display_mask=display_mask,
            display_keypoints=display_keypoints,
            return_figure=True,
            width=width if width < height-50 else None,
            height=height-50 if height-50 < width else None
        )
        return img
    
    # add interactions
    def _next(_):
        nonlocal current
        nonlocal records
        try:
            index = int(current.value)
            if index == len(records):
                index = 1
            else:
                index += 1
            current.value = str(index)
        except:
            pass
        
            
    def _previous(_):
        nonlocal current
        nonlocal records
        try:
            index = int(current.value)
            if index == 1:
                index = len(records)
            else:
                index -= 1
            current.value = str(index)
        except:
            pass
        
    btn_prev.on_click(_previous)
    btn_next.on_click(_next)
    
    return pn.Column(gui, pn.Row(_plot, align="center"))

In [ ]:
gallery(test_valid_records, height=500, width=300)

## Class histogram

In [ ]:
#export
def barplot_class_occurance(record_stats, class_map=None, width=500, height=500):
    """Creates a barplot of the class occurances."""
    result = record_stats.groupby("label").count()["id"]
    counts, values = result.values, result.index.tolist()
    # sort result in decending order
    fig = barplot(counts, values, class_map=class_map, width=width, height=height)
    return pn.Column(fig)

In [ ]:
barplot_class_occurance(test_record_dataset.data)

In [ ]:
barplot_class_occurance(test_record_dataset_no_class_map.data)

## Area histogram

In [ ]:
#export
def area_histogram(record_stats, class_label, bins=10, normalized=False, density=False, range=None, width=500, height=500):
    """Creates a histogram for a given class_label from record_stats."""
    if normalized:
        df_col = "area_normalized"
        x_label = "Area (normalized)"
    else:
        df_col = "area"
        x_label = "Area"
    title = f"{class_label}: {record_stats[record_stats['label'] == class_label].size} objects"   
    
    p = figure(width=width, height=height, x_axis_label = x_label, y_axis_label = "No. annotations", title=title)
    p = histogram(record_stats[record_stats["label"] == class_label][df_col].values, density=density, bins=bins, range=range, plot_figure=p, width=500, height=500)
    return p

In [ ]:
pn.Row(area_histogram(test_record_dataset.data, "can", range=(0, 45000), bins=20, density=True))

In [ ]:
pn.Row(area_histogram(test_record_dataset_no_class_map.data, 1))

In [ ]:
#export
def comparison_histogram_with_gui(record_dataset_list, hist_func, width=500, height=500):
    """Creates histograms for a list of record_stats and a histogram function (based on the histogram function from utils) with a full gui to customize the histogram parameters. 
    The hist function must have the following call head: (record_stats, class_label, bins, normalized, density, range, width, height)"""
    # gui
    # remove the first entry from the class map, because the background class is not explicit part of the annotaitons
    unique_labels = sorted(pd.unique(record_dataset_list[0].data["label"]))
    for record_dataset in record_dataset_list[1:]:
        if not sorted(pd.unique(record_dataset.data["label"])) == unique_labels:
            raise ValueError("All dataframes in the records_stats_list need to have the same set of unique values.")
    options = pd.unique(record_dataset_list[0].data["label"])
    options.sort()
    class_dropdown = pnw.Select(options=options.tolist())
    
    bins_slider = pnw.IntSlider(name="Bins", start=10, end=100, step=5)
    checkbox_normalized = pnw.Checkbox(name="Normalized", value=False)
    checkbox_density = pnw.Checkbox(name="Density", value=False)
    
    range_min = int(min(record_stats.data["area"].min() for record_stats in record_dataset_list))
    range_max = int(max(record_stats.data["area"].max() for record_stats in record_dataset_list))
    steps = (range_max-range_min)/50
    range_slider = pnw.RangeSlider(name="Range", start=range_min, end=range_max, step=steps)
    
    @pn.depends(class_dropdown.param.value, bins_slider.param.value, checkbox_normalized.param.value, checkbox_density.param.value, range_slider.param.value)
    def _draw_histogram(class_label, bins, normalized, density, range):
        nonlocal width
        nonlocal height
        nonlocal record_dataset_list
        # make the range slider dynamic with respect to the normalization
        if normalized:
            if range_slider.end != 1:
                old_slider_end = range_slider.end
                old_slider_start = range_slider.start
                range_slider.start = 0
                range_slider.end = 1
                range_slider.step = 1/50
                range_slider.value = ((range_slider.value[0]-old_slider_start)/(old_slider_end-old_slider_start), (range_slider.value[1]-old_slider_start)/(old_slider_end-old_slider_start))
                range = range_slider.value
        else:
            if range_slider.end != int(max(record_stats.data["area"].max() for record_stats in record_dataset_list)):
                range_slider.start = int(min(record_stats.data["area"].min() for record_stats in record_dataset_list))
                range_slider.end = int(max(record_stats.data["area"].max() for record_stats in record_dataset_list))
                range_slider.step = (range_max-range_min)/50
                range_slider.value = ((range_slider.value[0]*(range_slider.end-range_slider.start))+range_slider.start, (range_slider.value[1]*(range_slider.end-range_slider.start))+range_slider.start)
                range = range_slider.value
        return pn.Row(*[hist_func(record_stats.data, class_label, bins=bins, normalized=normalized, density=density, range=range, width=width, height=height) for record_stats in record_dataset_list])
    
    return pn.Column(class_dropdown, pn.Row(bins_slider, checkbox_normalized), pn.Row(range_slider, checkbox_density), _draw_histogram)

In [ ]:
comparison_histogram_with_gui([test_record_dataset, test_record_dataset], area_histogram)

In [ ]:
comparison_histogram_with_gui([test_record_dataset_no_class_map, test_record_dataset_no_class_map, test_record_dataset_no_class_map], area_histogram)

In [ ]:
#export
def annotations_per_image_stacked_hist(record_stats: pd.DataFrame, width=500, height=500):
    #generate tooltip
    tooltip = [(class_name, "@"+class_name) for class_name in record_stats.label.unique().astype(str)]
    hover = [("Annotations: ", "@can")]
    
    p = figure(title="Annotations per Image", x_axis_label="Number of annoations in the image", y_axis_label="Occurance", tooltips=tooltip, width=width, height=height)
    annotation_classes = record_stats["label"].unique().tolist()
    x = list(range(1, max(record_stats["num_annotations"]+1)))
    # if there are more than 256 classes make the color loop the color palette
    num_loops = len(annotation_classes)//256
    last_loop_residual = len(annotation_classes)%256
    colors = [*turbo(256)*num_loops, *turbo(last_loop_residual)]
    #collect data for the plot
    data = {"num_annotations": x}
    for label in annotation_classes:
        num_annotations = record_stats.groupby("label").get_group(label).groupby("num_annotations").count()["id"]
        values = np.zeros(max(x))
        for index, value in num_annotations.items():
            values[index-1] = value
        data[str(label)] = values
    annotation_classes = [str(entry) for entry in annotation_classes]
    p.vbar_stack(annotation_classes, x="num_annotations", width=0.9, color=colors, source=data, legend_label=annotation_classes)
    p.legend.click_policy="hide"

    return p

In [ ]:
pn.Row(annotations_per_image_stacked_hist(test_record_dataset.data))

In [ ]:
pn.Row(annotations_per_image_stacked_hist(test_record_dataset_no_class_map.data))